# Hybrid Recommendation System
Combining Content-Based Filtering and Collaborative Filtering using SVD.


In [ ]:
import pandas as pd
from surprise import SVD, Dataset, Reader, accuracy
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from surprise.model_selection import train_test_split

# Load datasets
ratings = pd.read_csv('ratings.csv')
restaurants = pd.read_csv('restaurants.csv')

# Collaborative Filtering
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(ratings[['user_id', 'restaurant_id', 'rating']], reader)
trainset, testset = train_test_split(data, test_size=0.2)
algo = SVD()
algo.fit(trainset)
predictions = algo.test(testset)
print('SVD RMSE:', accuracy.rmse(predictions))

# Content-Based Filtering
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(restaurants['description'])
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

def hybrid_recommendation(user_id, restaurant_id, alpha=0.5):
    # CF Prediction
    cf_pred = algo.predict(user_id, restaurant_id).est
    # CBF Prediction
    sim_scores = cosine_sim[restaurant_id] if restaurant_id < len(cosine_sim) else [0]*len(cosine_sim)
    cbf_pred = sum(sim_scores) / len(sim_scores)
    # Hybrid Prediction
    return alpha * cf_pred + (1 - alpha) * cbf_pred

print('Hybrid Recommendation for user 1 and restaurant 10:', hybrid_recommendation(1, 10))
